In [1]:
import sys
sys.path.append('..')
import pandas as pd
from Constants import Datasets
import numpy as np
datasets = [dataset.name for dataset in Datasets]
metrics = [
'total_num_words',
'avg_word_length', 
'total_num_sent',
'avg_sent_length',
'token_type_ratio',
'symbol_word_ratio',
'num_non_alphabet_words'
]
num_samples = 10000

In [2]:
#embeddings dim reduced
points = pd.DataFrame()
for i, d1 in enumerate(datasets):
    d = pd.DataFrame(np.random.randn(num_samples, 2) - i, columns=['pc1', 'pc2'])
    d['name'] = d1
    points = pd.concat((points, d))
import uuid
points['id'] = [uuid.uuid4() for _ in range(len(points))]
points['lang'] = 'de'

In [3]:
#generate dummy metrics for datasets
all_metrics = pd.DataFrame()
for j, dataset in enumerate(datasets):
    metrics_for_dataset = pd.DataFrame(np.random.randn(num_samples, len(metrics)) + j, columns=metrics)
    all_metrics = pd.concat((all_metrics, metrics_for_dataset))
all_metrics['id'] = points['id']

In [4]:
points.to_csv('../datasets/dataset.csv', index=False)
all_metrics.to_csv('../datasets/metrics.csv', index=False)

In [5]:
#generate composition details 
import pandas as pd
df = pd.read_csv("../datasets/dataset.csv")
comp = df.groupby(by=['name', 'lang']).count().reset_index()[['name', 'lang', 'id']]
comp.columns=['name', 'lang', 'count']
comp.to_csv('../datasets/global_size.csv', index=False)

In [9]:
#test composition for filtered dataset
import pandas as pd
df = pd.read_csv("../datasets/dataset.csv")


,pc1,pc2,name,id,lang
0,-1.240151,-0.801965,WIKI40B,474ebb8f-f642-4b8c-82cb-94c90319f742,de
1,1.027551,0.248243,WIKI40B,bc65670d-e86b-45b1-9d1b-7c819b2fac0c,de
2,-2.281245,-0.532618,WIKI40B,01a97ee5-31ef-4dcb-b0e8-62861f92ec81,de
3,0.570188,0.696386,WIKI40B,a72686fe-7514-4177-b147-5d152545b552,de
4,0.215394,-0.888787,WIKI40B,d74a5b1c-6ca7-400b-9eb5-8603cd6558f5,de
...,...,...,...,...,...
39995,-2.012062,-1.883281,ONE_MILLION_POSTS,dc9e85ae-6c75-44e9-b8c4-efb136651549,de
39996,-2.368473,-4.270414,ONE_MILLION_POSTS,cb1d21dd-651b-4a77-9c17-fb233b5a4bc2,de
39997,-1.360764,-2.085249,ONE_MILLION_POSTS,a63b09dd-9819-4984-aabe-fbb5f9dc6965,de
39998,-3.978291,-2.768367,ONE_MILLION_POSTS,1c68dc6a-996d-45d0-a15f-9ad9f07239c6,de


,name,lang,id
0,BUNDESTAG,de,10000
1,ONE_MILLION_POSTS,de,10000
2,OPENSUBTITLES,de,10000
3,WIKI40B,de,10000


In [18]:
import plotly.express as px
fig = px.scatter(x=[0, 1, 2, 3, 4], y=[0, 1, 4, 9, 16])
fig.show()

In [38]:
import plotly.graph_objects as go
# fig = px.scatter(x=[0, 1, 2, 3, 4], y=[0, 1, 4, 9, 16])
fig.add_trace(go.Scatter(x=[7,2], y=[2,3],
                    mode='markers', name='d', hover_data='s'))
fig.show()